# Load packages

In [ ]:
library(ArchR)
library(Giotto)
library(parallel)

In [ ]:
addArchRThreads(threads <- 20)
addArchRGenome("mm10")

# Set dictionary

In [ ]:
data_dir = "spatial_CUT_Tag/H3K4me3_E11_50um/processed_data/"
save_dir = "H3K4me3_E11_50um/"

# Load data

In [ ]:
tissue_positions_path = paste0(data_dir, 'spatial/tissue_positions_list.csv')
fregment = paste0(data_dir, 'fragments.tsv.gz')
png_path = paste0(data_dir, 'spatial/tissue_lowres_image.png')

# Create ArchR arrow files

In [ ]:

ArrowFiles <- createArrowFiles(inputFiles = c(fregment), sampleNames = c('H3K4me3'),
                               filterTSS = 0, filterFrags = 0, minFrags = 0, maxFrags = 1e+07,
                               addTileMat = TRUE, addGeneScoreMat = TRUE, offsetPlus =0, offsetMinus=0,
                               TileMatParams = list(tileSize = 5000), force = TRUE)

H3K4me3_all <- ArchRProject(ArrowFiles, outputDirectory = save_dir, copyArrows = TRUE)

In [ ]:
## spatial locations and get in-tissue spots
spatial_results = data.table::fread(tissue_positions_path)
spatial_results[, V1 := paste0('H3K4me3#',V1,'-1'), by = 1:nrow(spatial_results)]
spatial_results = spatial_results[match(H3K4me3_all$cellNames, V1)]
colnames(spatial_results) = c('barcode', 'in_tissue', 'array_row', 'array_col', 'col_pxl', 'row_pxl')
#
cellNames_in_tissue = spatial_results[in_tissue==1,]$barcode
H3K4me3 = H3K4me3_all[cellNames_in_tissue,]
spatial_locs = spatial_results[match(H3K4me3$cellNames, barcode),.(barcode,row_pxl,-col_pxl)]
data.table::setnames(spatial_locs, new=c("barcode","x","y"))
spatial_locs = data.frame(spatial_locs)
rownames(spatial_locs) = spatial_locs$barcode
spatial_locs = spatial_locs[rownames(H3K4me3@cellColData),]


## Ploting statistics

In [ ]:
df <- getCellColData(H3K4me3, select = c("log10(nFrags)", "TSSEnrichment"))
p <- ggPoint(
 x = df[,1],
    y = df[,2],
    colorDensity = TRUE,
    continuousSet = "sambaNight",
    xlabel = "Log10 Unique Fragments",
    ylabel = "TSS Enrichment",
    xlim = c(log10(500), quantile(df[,1], probs = 0.99)),
    ylim = c(0, quantile(df[,2], probs = 0.99))
) + geom_hline(yintercept = 4, lty = "dashed") + geom_vline(xintercept = 3, lty = "dashed")
p
png(paste0(save_dir,"/1.TSS-vs-Frags.png"),width = 300, height = 300)
plot(p)
dev.off()


p1 <- plotGroups(ArchRProj = H3K4me3, groupBy = "Sample", colorBy = "cellColData",
    name = "TSSEnrichment", plotAs = "ridges",baseSize=24)
p1
png(paste0(save_dir,"/1.TSS-rideges.png"))
plot(p1)
dev.off()

p2 <- plotGroups(ArchRProj = H3K4me3, groupBy = "Sample", colorBy = "cellColData",
    name = "TSSEnrichment", plotAs = "violin", alpha = 0.4, addBoxPlot = TRUE, baseSize=20)
p2
png(paste0(save_dir,"/1.TSS-violin.png"),width = 200, height = 500)
plot(p2)
dev.off()

p1 <- plotFragmentSizes(ArchRProj = H3K4me3)
p1
png(paste0(save_dir,"/2.Fragment-size-distribution.png"))
plot(p1)
dev.off()


p2 <- plotTSSEnrichment(ArchRProj = H3K4me3)
p2
png(paste0(save_dir,"/2.TSS-enrichment-profiles.png"))
plot(p2)
dev.off()

# Iterative Latent Semantic Indexing (LSI)

In [ ]:
H3K4me3 <- addIterativeLSI(ArchRProj = H3K4me3, 
                           useMatrix = "TileMatrix", 
                           name = "IterativeLSI", 
                           iterations = 3, 
                           clusterParams = list(resolution = c(0.2), 
                                                sampleCells = 10000,
                                                n.start = 10), 
                            varFeatures = 25000, 
                           dimsToUse = 1:30
                          )

# UMAP

In [ ]:
H3K4me3 <- addUMAP(ArchRProj = H3K4me3, 
                   reducedDims = "IterativeLSI", 
                   name = "UMAP", 
                   nNeighbors = 20, 
                   minDist = 0.4, 
                   metric = "cosine"
                  )

# Clustering

In [ ]:
H3K4me3 <- addClusters(input = H3K4me3, 
                       reducedDims = "IterativeLSI", 
                       method = "Seurat", 
                       name = "Clusters",
                       resolution = 1
                      )

In [ ]:
p1 <- plotEmbedding(ArchRProj = H3K4me3, colorBy = "cellColData", 
                    name = clus, embedding = "UMAP",
                    size = 2, baseSize = 8)

## Save data

In [ ]:
save(H3K4me3, file = paste0(save_dir, '/H3K4me3.RData'))